In [222]:
import pandas as pd
import numpy as np
import random
import math

In [227]:
class ClassImbalance:
    def __init__(self):
        return

#     def read_data(self, filepath):
#         colums = ["Sex","Length","Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", 
#                     "Shell_weight", "Class"]
#         data = pd.read_csv(filepath, names=colums)
#         data = pd.DataFrame(data)
#         self.data = data
#         return

#     def pre_process(self):
#         #check for imbalance

#         label = self.data["Class"]
#         features = self.data.drop(["Class"], axis=1)
    
#         print("Label length: ", len(label))

#         positive_label = len(label.loc[label != "negative"])
#         negative_label = len(label.loc[label == "negative"])
        
#         #th
        
#         features = label.loc[label != "negative"]
#         return features
    def read_data(self, filepath):
        colums = ["Sex","Length","Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", 
                    "Shell_weight", "Class"]
        data = pd.read_csv(filepath, names=colums)
        data = pd.DataFrame(data)
#         data["Class"] = data["Class"].str.strip()
        self.data = np.array(data)
        for i in range(self.data.shape[0]):
            self.data[i, 8] = self.data[i, 8].strip()
    
        return

    def pre_process_undersample(self, k, label):

        undersampled_data = np.array([])
        label_index = []

        #select indexes of rows with specified label
        for row in range(self.data.shape[0]):
            if self.data[row, 8] == label:
                label_index.append(row)

        random_remove = random.sample(label_index, k)

        for i in range(self.data.shape[0]):
            if i not in random_remove:
                if len(undersampled_data) == 0:
                    undersampled_data = self.data[i, :].copy()
                else:
                    undersampled_data = np.vstack((undersampled_data, self.data[i, :]))

        return undersampled_data

    def pre_process_oversample(self, k, label):
        oversampled_data = self.data.copy()
        label_index = []

        for row in range(self.data.shape[0]):
            if self.data[row, 8] == label:
                label_index.append(row)

        for i in range(k):
            index = label_index[random.randint(0, (len(label_index)-1))]
            item = self.data[index]

            oversampled_data = np.vstack((oversampled_data, item))
        
        return oversampled_data

    def euclidean_distance(self, point1, point2):
        print(point1)
        print(point2)
        total = 0
#         if type(point1) == str:
#             point1 = float(point1)
#         if type(point2) == str:
#             point2 = float(point2)   
            
        for i in range(len(point1)):
            total = total + pow(point1[i] + point2[i], 2)

        return math.sqrt(total)

    def smote(self, k, label):
        data = self.data.copy()
        label_index = []

        for row in range(self.data.shape[0]):
            if self.data[row, 8] == label:
                label_index.append(row)
        
        for i in range(k):
            rand_item = label_index[random.randint(0, (len(label_index) - 1))]

            #KNN
            distances = []
            distance_index = []

            for item in label_index:
                if item == rand_item:
                    continue
                
                distances.append(self.euclidean_distance(self.data[item], self.data[rand_item]))
                distance_index.append(item)

            k_neigh = []

            for neigh in range(2):
                nearest = np.argmin(distances)
                k_neigh.append(distance_index[nearest])
                distances[nearest] = float("inf")
            
            neigh_rand = k_neigh[random.randint(0, (len(k_neigh) - 1))]

            feat = []
            for j in range(7):
                col = self.data[neigh_rand, j] - self.data[rand_item, j]
                feat.append(col)
            
            alpha = random.random()

            all_feat = []

            for j in range(len(feat)):
                col = self.data[rand_item, j] + alpha*feat[j]
                all_feat.append(col)
            
            all_feat.append(label)

            data = np.vstack((data, all_feat))

        return data

In [228]:
abalone = ClassImbalance()
abalone.read_data("/Users/oyinlola/Desktop/MSc Data Science/SCC403 - Data Mining/Coursework/abalone19.csv")


negative, positive = 0, 0
for i in range(abalone.data.shape[0]):
    if abalone.data[i, 8] == "negative":
        negative += 1
    elif abalone.data[i, 8] == "positive":
        positive += 1

print("Negative: ", negative)
print("Positive: ", positive)

Negative:  4142
Positive:  32


In [229]:
undersampled = abalone.pre_process_undersample(4110, "negative")
oversampled = abalone.pre_process_oversample(4110, "positive")


In [231]:
# smoted = abalone.smote(4110, "positive")

In [218]:
len(undersampled)
undersampled

array([['F', 0.55, 0.44, 0.15, 0.8945, 0.3145, 0.151, 0.32, 'positive'],
       ['F', 0.68, 0.55, 0.175, 1.798, 0.815, 0.3925, 0.455, 'positive'],
       ['F', 0.4, 0.32, 0.11, 0.353, 0.1405, 0.0985, 0.1, 'negative'],
       ['M', 0.595, 0.465, 0.175, 1.115, 0.4015, 0.254, 0.39, 'negative'],
       ['M', 0.7, 0.535, 0.16, 1.7255, 0.63, 0.2635, 0.54, 'positive'],
       ['M', 0.56, 0.45, 0.185, 1.07, 0.3805, 0.175, 0.41, 'positive'],
       ['F', 0.635, 0.505, 0.17, 1.415, 0.605, 0.297, 0.365, 'negative'],
       ['F', 0.565, 0.455, 0.175, 1.013, 0.342, 0.207, 0.35, 'positive'],
       ['M', 0.595, 0.455, 0.195, 1.3305, 0.4595, 0.3235, 0.345,
        'positive'],
       ['M', 0.63, 0.515, 0.16, 1.016, 0.4215, 0.244, 0.355, 'positive'],
       ['F', 0.66, 0.565, 0.195, 1.7605, 0.6920000000000001, 0.3265, 0.5,
        'negative'],
       ['F', 0.68, 0.58, 0.2, 1.787, 0.585, 0.45299999999999996, 0.6,
        'positive'],
       ['M', 0.6, 0.5, 0.16, 1.015, 0.3995, 0.1735, 0.33, 'positive']

In [220]:
len(oversampled)
oversampled

array([['M', 0.455, 0.365, ..., 0.10099999999999999, 0.15, 'negative'],
       ['M', 0.35, 0.265, ..., 0.0485, 0.07, 'negative'],
       ['F', 0.53, 0.42, ..., 0.1415, 0.21, 'negative'],
       ...,
       ['M', 0.6, 0.5, ..., 0.1735, 0.33, 'positive'],
       ['I', 0.45, 0.355, ..., 0.0665, 0.16, 'positive'],
       ['M', 0.5, 0.38, ..., 0.1365, 0.215, 'positive']], dtype=object)

In [203]:
abalone.data[2200, 8]

'positive'

In [232]:
df = pd.DataFrame(abalone.data)

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4174 entries, 0 to 4173
Data columns (total 9 columns):
0    4174 non-null object
1    4174 non-null object
2    4174 non-null object
3    4174 non-null object
4    4174 non-null object
5    4174 non-null object
6    4174 non-null object
7    4174 non-null object
8    4174 non-null object
dtypes: object(9)
memory usage: 293.6+ KB


In [244]:
df.describe()

,0,1,2,3,4,5,6,7,8
count,4174,4174.000,4174.00,4174.00,4174.0000,4174.000,4174.0000,4174.000,4174
unique,3,134.000,111.00,51.00,2428.0000,1514.000,880.0000,926.000,2
top,M,0.625,0.45,0.15,0.2225,0.175,0.1715,0.275,negative
freq,1526,94.000,139.00,267.00,8.0000,11.000,15.0000,43.000,4142


In [242]:
new_df = df.drop([0], axis=1)

In [247]:
new_df[1].describe()

count     4174.000
unique     134.000
top          0.625
freq        94.000
Name: 1, dtype: float64